# Q_ RATE OF CHANGE

In [21]:
from koselleck import *

In [ ]:
m1,m2=get_ameasure_change_over_decades

In [23]:
def do_measure_change_over_decades(obj,words=None,progress=True,**y):
    try:
        row1,row2=obj
        if words is None: words=get_valid_words()
        m1,m2=load_model_row(row1),load_model_row(row2)
        odf=measure_change(m1,m2,words=words,progress=progress,
#                            funcs=[measure_change_noiseaware],
                           **y).reset_index()
        odf['period1'],odf['period2']=row1.period,row2.period
        return odf
    except Exception:
        return pd.DataFrame()

def measure_change_over_decades(dfgrp,words=None,num_proc=1,progress=True,**y):
    dfgrp=dfgrp.sort_values('period_start')
    row1=None
    m1=None
    odf=pd.DataFrame()
    #iterr=tqdm(dfgrp.iterrows(),total=len(dfgrp)) if progress else dfgrp.iterrows()
    rows=[row for i,row in dfgrp.iterrows()]
    objs=[
        (row1,row2)
        for i1,row1 in dfgrp.iterrows() 
        for i2,row2 in dfgrp.iterrows() 
        if row2.period>row1.period
    ]
#     objs=random.sample(objs,3)
    resdfs=[
        x for x in pmap(
            do_measure_change_over_decades,
            objs,
            num_proc=num_proc,
            progress=progress,
            kwargs=dict(
                progress=False,
                words=words,
                **y
            )
        ) if type(x)==pd.DataFrame
    ]
    return pd.concat(resdfs) if len(resdfs) else pd.DataFrame()

In [24]:
dfdecs=get_pathdf_models_bydecade()
grps=dfdecs.groupby('run')
for i,dfgrp in grps: pass
# dfgrp

In [25]:
# gen
odf=measure_change_over_decades(dfgrp,num_proc=6)
# save
odf.to_pickle('data/data.semantic_change_over_decades.1run.v2.pkl')
odf

Mapping do_measure_change_over_decades() [x6]: 100%|██████████| 190/190 [1:51:18<00:00, 35.15s/it]  


,word,rank,perc,dist,neighborhood1_local,neighborhood2_local,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,...,source_abstractness,period_abstractness,score_diff_abstractness,z_abstractness,is_clean_noiseaware,z_noiseaware,z_procrustes,z_local,period1,period2
0,repose,3.25,99.924344,2.011760,"-cloak, -chariot, -dun, -velvet, -mule, -bride...","+safety, +welfare, +tranquillity, +prosperity,...",1.0,3.0,5.0,4.0,...,Median,orig,2.575308,6.290421,False,3.916678,3.962273,8.368451,1700-1710,1710-1720
1,agent,10.75,99.672159,1.447165,"-rebel, -adultery, -tribute, -digest, -resigna...","+villa, +churchyard, +gel, +abbey, +raid, +bal...",6.0,11.0,13.0,13.0,...,Median,orig,-1.865772,4.211257,True,3.610455,3.641559,5.391975,1700-1710,1710-1720
2,heft,18.75,99.403161,1.406349,"-honey, -beef, -site, -tag, -bacon, -pine, -ba...","+moderation, +purity, +advancement, +schism, +...",4.0,31.0,24.0,16.0,...,Median,orig,1.872837,4.231960,True,2.875589,2.954555,4.519601,1700-1710,1710-1720
3,policy,19.75,99.369536,1.512577,"-ministry, -purity, -preservation, -institutio...","+allowance, +marriage, +payment, +zoo, +entran...",2.0,41.0,28.0,8.0,...,Median,orig,-2.005976,4.622100,False,2.583811,2.915612,6.381838,1700-1710,1710-1720
4,marriage,22.00,99.293880,1.057075,"-election, -lady, -chancellor, -throne, -princ...","+office, +policy, +payment, +benefit, +claim, ...",71.0,9.0,3.0,5.0,...,Median,orig,-1.289671,2.523097,False,3.711755,3.975014,7.565473,1700-1710,1710-1720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,system,5838.25,2.533812,-0.228749,"principle, method, constitution, organization,...","principle, method, legislation, constitution, ...",5771.0,5950.0,5952.0,5680.0,...,Median,orig,0.007278,-0.966805,True,-1.182813,-1.184553,-0.476323,1870-1880,1890-1900
5985,table,5847.00,2.387711,-0.226005,"tray, shelf, stool, sideboard, sofa, mantelpie...","tray, sideboard, mantelpiece, sofa, shelf, cus...",5744.0,5900.0,5910.0,5834.0,...,Median,orig,-0.007830,-0.963023,True,-1.128902,-1.138327,-0.500592,1870-1880,1890-1900
5986,island,5871.75,1.974453,-0.234311,"mainland, coast, archipelago, peninsula, colon...","archipelago, coast, mainland, peninsula, promo...",5913.0,5870.0,5894.0,5810.0,...,Median,orig,-0.002493,-0.999605,True,-1.102564,-1.122401,-0.496588,1870-1880,1890-1900
5987,spot,5886.75,1.723994,-0.226630,"hillside, mound, cairn, beach, crag, shore, pr...","hillside, promontory, shore, mound, ground, pl...",5734.0,5926.0,5934.0,5953.0,...,Median,orig,0.008085,-0.961272,True,-1.155679,-1.157975,-0.530615,1870-1880,1890-1900


In [26]:
odf.groupby(['period1','period2']).size()

period1    period2  
1700-1710  1710-1720    2974
           1720-1730    2800
           1730-1740    3195
           1740-1750    3181
           1750-1760    3198
                        ... 
1860-1870  1870-1880    5980
           1880-1890    5980
           1890-1900    5980
1870-1880  1880-1890    5989
           1890-1900    5989
Length: 188, dtype: int64

In [5]:
# # gen
# odf2 = pmap_groups(
#     measure_change_over_decades,
#     dfdecs.groupby('run'),
#     num_proc=1,
#     kwargs=dict(progress=False,num_proc=6),
#     desc='Measuring semantic change over all decades and runs'
# ).reset_index()

# # save
# odf2.to_pickle('data/data.semantic_change_over_decades.runs.pkl')
# odf2